In [ ]:
!pip install google-auth google-auth-httplib2 google-auth-oauthlib google-api-python-client
!pip install -q -U google-generativeai
!pip install --upgrade google-generativeai

In [ ]:
#Import necessary libraries
from google.oauth2 import service_account
from googleapiclient.discovery import build

In [ ]:
import openpyxl
from openai import OpenAI
import os
import json
from tqdm import tqdm
import pandas as pd
import google.generativeai as genai

In [ ]:
# Import the API Key (insert key here)
GOOGLE_API_KEY=""
genai.configure(api_key=GOOGLE_API_KEY)

# Reading G-Drive Sheet

In [ ]:
!pip install --quiet gspread oauth2client google-auth
import gspread
from google.colab import auth
import pandas as pd
import google.auth

auth.authenticate_user()

# Authorize gspread client
# Use google.auth.default() to get credentials
credentials, project = google.auth.default()
gc = gspread.authorize(credentials)

def load_sheet(sheet_id, sheet_name="Sheet1"):
    # Load google sheet as dataframe
    worksheet = gc.open_by_key(sheet_id).worksheet(sheet_name)
    data = worksheet.get_all_values()
    df = pd.DataFrame(data[1:], columns=data[0])
    return df

#path blinded
path = ""
sheet = load_sheet(path,"roseWangResponses_mapped_20251112")

Get all the responses and scores

In [ ]:
INPUT_COLUMN = "response" # Specify the column from which input is read
sheet = sheet[(sheet["title"]=="Prompting Students to Explain") & (sheet["prompt"].str.startswith(("1.","9.")))]
sheet = sheet.reset_index(drop=True)
row_count = len(sheet) # Get number of rows.
inputs = [sheet.loc[i, INPUT_COLUMN] for i in range(row_count)]
inputs = [x for x in inputs if x is not None]
print(inputs)

# Gemini Prompt Setup

Scoring Prompt Input

In [ ]:
GEMINI_SYSTEM_PROMPT = """
You are a tutor evaluator. Please score the following tutor response to a tutor training scenario involving a middle school student:
-if the tutor’s response prompts the student to explain their thinking by asking open-ended questions without judgement and encouraging the student to reflect on their learning process, score with a 1. Sample responses scoring a 1 are "Hold on a second Ahan, I see you've got the right answer now, but I'd love to hear how you got there. Can you walk me through what made you change your first few answers and how you figured out this one?"; and “Say that the answer is right but then ask him how he got there.”; and "Now that you’ve got it, see if you can explain to me why your approach during your first attempt didn’t work."
-if the tutor’s response does not ask an open-ended question or does not encourage the student to reflect on their process or is judgemental, score with a 0. Sample responses scoring a 0 are "Ok that’s correct but how about I open the whiteboard and give you some pointers on how I would have solved it?" and "Did you get where you made the mistake during your first attempt?"; "It will be better if you try to verify the answer before submitting it."

Once given a response by the user, please return a JSON string following the format, {"Rationale": "your reasoning here", "Score":0/1}
"""

Helper function for response parsing

In [ ]:
def extract_response(response_obj, json=False):
  role = response_obj.choices[0].message.role
  content = response_obj.choices[0].message.content
  if json:
    return {"role": role, "content": content}
  else:
    return (role, content)

## gemini API Call

In [ ]:
# Iterate over all responses
MAX_TOKENS = 300
TEMPERATURE = 0
RUN_UP_TO = 300  # Sets a maximum index for responses to run. Useful to specify how many responses we want to run on (partial execution). Set to -1 to run them all.
SCORE_COLUMN = "response"  # Change column numbers here to  modify where output is written


MODEL = 'gemini-2.5-pro'
model = genai.GenerativeModel(MODEL, system_instruction=GEMINI_SYSTEM_PROMPT)

if RUN_UP_TO >=  0:  # If an upper bound is set
  inputs_upto = inputs[:RUN_UP_TO]
else:
  inputs_upto = inputs  # Take the whole set of responses
tempScoreList = []
tempDirectionList = []
tempRationaleList = []

for index, inpt in tqdm(enumerate(inputs_upto), total=len(inputs_upto)):
  generation_prompt = "Tutor Response: " + inpt + "\n\n. Your JSON: "
  generation_config = genai.GenerationConfig(temperature=TEMPERATURE)
  gemini_out = model.generate_content(generation_prompt, generation_config=generation_config)
  print(gemini_out)
  # Extract the content from the response
  content = gemini_out.text.lstrip("```json")[:-4]
  print(content)
  # We now need to parse the JSON into rational and score
  try:
    content_json = json.loads(content)  # Run response through JSON
    score = str(content_json["Score"])  # Cast to string to avoid type inequality
    rationale = str(content_json["Rationale"])  # Fetch the rationale
    sheet.at[index,"Gemini Score"] = score  # Now write both into the dataframe
    sheet.at[index,"Gemini Rationale"] = rationale
    tempScoreList.append(score)
    tempDirectionList.append(generation_prompt)
    tempRationaleList.append(rationale)

  except:
    print("error!")

print(tempScoreList)
print(tempDirectionList)
print(tempRationaleList)

In [ ]:
for i in range(len(inputs_upto)):
  print(inputs_upto[i])
  print(tempScoreList[i])
  print(tempRationaleList[i])
  print()

In [ ]:
print(list(sheet["Gemini Score"]))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Save G-Drive Sheet

In [ ]:
#path blinded
sheet.to_csv('', index=False)